In [66]:
import pandas as pd
import keras 

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
#Para la LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [57]:
df = pd.read_csv("Datasets/Duplicated5MeanPastMatches.csv")
df = df.dropna()

In [58]:
df

,surface,player1_hand,player2_hand,player1_age,player2_age,best_of,tourney_date,player1_ht,player1_ace,player1_df,...,player2_svpt,player2_1stIn,player2_1stWon,player2_2ndWon,player2_SvGms,player2_bpSaved,player2_bpFaced,player2_rank,player2_rank_points,label
0,Hard,R,L,21.7,31.1,3,20000110,188.0,6.25,3.75,...,88.80,48.00,34.8,20.40,12.40,7.80,9.80,74.0,638.2,0
1,Hard,R,L,24.5,24.3,3,20000110,190.0,6.40,4.00,...,74.60,38.60,25.4,17.80,11.60,4.60,8.40,48.0,890.4,0
2,Hard,R,R,21.3,26.5,3,20000110,175.0,1.00,1.20,...,83.60,66.00,38.4,8.80,12.00,6.00,10.20,80.0,598.0,0
3,Hard,R,R,19.9,18.4,3,20000110,183.0,1.60,2.20,...,73.20,41.60,30.8,15.80,11.00,3.00,5.20,82.6,588.2,0
4,Hard,R,R,27.3,23.7,3,20000110,180.0,1.00,2.50,...,87.20,49.80,39.0,18.80,13.00,5.20,7.20,52.4,868.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139570,Hard,R,R,25.4,19.7,3,20230203,178.0,3.00,2.00,...,69.60,42.80,32.0,12.80,10.20,4.20,5.60,7.6,3788.0,1
139593,Hard,R,R,28.3,26.0,3,20230916,191.0,1.00,4.00,...,96.00,70.00,48.0,15.00,15.00,5.00,7.00,431.6,150.2,1
139598,Clay,R,R,18.4,27.1,3,20230916,193.0,6.00,2.00,...,99.25,64.25,43.0,17.00,15.25,5.75,10.25,279.2,243.6,1
139599,Clay,R,R,31.7,27.0,3,20230915,185.0,1.40,3.80,...,95.50,55.25,34.5,18.25,13.75,6.00,11.25,447.2,79.4,1


In [59]:
df_one_hot = pd.get_dummies(df, columns=["surface", "player1_hand", "player2_hand", "best_of"], dtype=int)

In [61]:
X = df_one_hot.drop(["label"],axis = 1)
y = df_one_hot["label"]

In [62]:
X_train, X_test = X[X["tourney_date"] < 20150101], X[X["tourney_date"]>= 20150101]
y_train, y_test = y[X_train.index], y[X_test.index]

X_train, X_test = X_train.drop("tourney_date", axis=1), X_test.drop("tourney_date", axis=1)

In [67]:
model = Sequential()
model.add(Dense(128, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(128, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="models\Full",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

history = model.fit(X_train, y_train, verbose=1, epochs=1000, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 1/1000
2691/2691 [==============================] - 9s 3ms/step - loss: 0.9156 - accuracy: 0.6395 - val_loss: 0.6742 - val_accuracy: 0.6430 - lr: 0.0010
Epoch 2/1000
2691/2691 [==============================] - 9s 3ms/step - loss: 0.6194 - accuracy: 0.6630 - val_loss: 0.6562 - val_accuracy: 0.6435 - lr: 0.0010
Epoch 3/1000
2691/2691 [==============================] - 8s 3ms/step - loss: 0.6126 - accuracy: 0.6636 - val_loss: 0.6385 - val_accuracy: 0.6417 - lr: 0.0010
Epoch 4/1000
2691/2691 [==============================] - 8s 3ms/step - loss: 0.6070 - accuracy: 0.6685 - val_loss: 0.6274 - val_accuracy: 0.6415 - lr: 0.0010
Epoch 5/1000
2691/2691 [==============================] - 8s 3ms/step - loss: 0.6051 - accuracy: 0.6673 - val_loss: 0.6347 - val_accuracy: 0.6424 - lr: 0.0010
Epoch 6/1000
2691/2691 [==============================] - 8s 3ms/step - loss: 0.6038 - accuracy: 0.6689 - val_loss: 0.6274 - val_accuracy: 0.6408 - lr: 0.0010
Epoch 7/1000
2691/2691 [======================

KeyboardInterrupt: 